# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [6]:
!pip install langchain_community

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=None)


In [9]:
prompt = ChatPromptTemplate.from_template( 'Give a list 3 sumarised reviews and general sentiment evaluation {data}')
 


In [10]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\Sadiq\AppData\Local\Temp\ipykernel_20852\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [11]:
product = 'Luxury Air Mattress'
chain.run(product)

C:\Users\Sadiq\AppData\Local\Temp\ipykernel_20852\723715470.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(product)


'1. Review 1: "The luxury air mattress is a game changer! It\'s incredibly comfortable and durable, providing a restful night\'s sleep. The built-in pump is convenient and inflates the mattress quickly. Highly recommend for those looking for a high-quality sleeping solution."\n\n2. Review 2: "I was skeptical about purchasing a luxury air mattress, but I have been pleasantly surprised by the outstanding comfort and support it offers. The material is top-notch and feels luxurious. The price may be a bit high, but the quality and performance make it worth every penny."\n\n3. Review 3: "I\'ve tried many air mattresses in the past, but this luxury model stands out above the rest. The plush surface feels like sleeping on a cloud, and the firmness can be adjusted to suit individual preferences. Overall, I am extremely satisfied with this purchase and would highly recommend it." \n\nGeneral sentiment evaluation: The luxury air mattress generally receives positive reviews, with users praising i

## SimpleSequentialChain

In [12]:
from langchain.chains import SimpleSequentialChain

In [13]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    ' Give a the top 5 product sentsentiments from {data}'
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [14]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    'Give a table of the top 5 product sentiments and their respective counts {data}'
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [15]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [16]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
1. "This luxurious air mattress exceeded all of my expectations in terms of comfort and durability."
2. "The premium materials used in this air mattress make it feel like sleeping on a cloud."
3. "The adjustable firmness settings on this air mattress allowed me to customize my sleeping experience to perfection."
4. "I've never slept better than I have on this luxury air mattress - worth every penny."
5. "The sophisticated design of this air mattress adds a touch of elegance to my bedroom decor."
| Sentiment                                                                                           | Count |
|-----------------------------------------------------------------------------------------------------|-------|
| Exceeded expectations in comfort and durability                                                     | 1     |
| Feels like sleeping on a cloud thanks to premium materials                                            | 1     |
|

'| Sentiment                                                                                           | Count |\n|-----------------------------------------------------------------------------------------------------|-------|\n| Exceeded expectations in comfort and durability                                                     | 1     |\n| Feels like sleeping on a cloud thanks to premium materials                                            | 1     |\n| Adjustable firmness settings allow for customized sleeping experience                                 | 1     |\n| Best sleep ever on this luxury air mattress, worth the investment                                     | 1     |\n| Sophisticated design adds elegance to bedroom decor                                                   | 1     |'

**Repeat the above twice for different products**

## SequentialChain

In [17]:
from langchain.chains import SequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  ' Give a the top 5 product sentsentiments from {data}'
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='English'
                    )


In [19]:
second_prompt = ChatPromptTemplate.from_template(
  'Give a table of the top 5 product sentiments and their respective counts {data}'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='English_0'
                    )


In [20]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    'translate to Yoruba to french {data}'
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='English_1'
                      )


In [21]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        'translate to Hausa: {data}'
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='French'
                     )


In [22]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain

In [23]:
pip show langchain

Name: langchain
Version: 0.3.27
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: c:\users\sadiq\onedrive\documents\anaconda3\envs\new_hack_lib\lib\site-packages
Requires: async-timeout, langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community
Note: you may need to restart the kernel to use updated packages.


In [25]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["data"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

ValidationError: 1 validation error for SequentialChain
  Value error, Expected output variables that were not found: {'English_Review', 'followup_message', 'summary'}. [type=value_error, input_value={'chains': [LLMChain(verb...sage'], 'verbose': True}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

In [26]:
from langchain.chains import LLMChain, SequentialChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

# 1) Make clear English from raw Review
p1 = PromptTemplate(
    input_variables=["Review"],
    template="Rewrite the following review in clear English:\n\n{Review}"
)
chain_one = LLMChain(llm=llm, prompt=p1, output_key="English")

# 2) Summarize the English review
p2 = PromptTemplate(
    input_variables=["English"],
    template="Summarize this review in 2 sentences:\n\n{English}"
)
chain_two = LLMChain(llm=llm, prompt=p2, output_key="Summary")

# 3) Draft a follow-up message
p3 = PromptTemplate(
    input_variables=["English", "Summary"],
    template=("Write a polite follow-up message to the reviewer, "
              "using the English text and its summary.\n\n"
              "English:\n{English}\n\nSummary:\n{Summary}")
)
chain_three = LLMChain(llm=llm, prompt=p3, output_key="Followup")

# 4) Translate the English review to French
p4 = PromptTemplate(
    input_variables=["English"],
    template="Translate to French:\n\n{English}"
)
chain_four = LLMChain(llm=llm, prompt=p4, output_key="French")

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],                   # <- name used when calling the chain
    output_variables=["English", "Summary", "Followup", "French"],  # <- unique, real keys
    verbose=True
)

# Run
result = overall_chain({"Review": "orig review text here"})
print(result["English"])
print(result["Summary"])
print(result["Followup"])
print(result["French"])


C:\Users\Sadiq\AppData\Local\Temp\ipykernel_20852\1568208738.py:45: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = overall_chain({"Review": "orig review text here"})




> Entering new SequentialChain chain...

> Finished chain.
Sure! Please provide the review you would like me to rewrite.
Please share the review you'd like summarized, and I'll help you with that!
Subject: Follow-Up on Your Review Request

Dear [Reviewer's Name],

I hope this message finds you well. I wanted to follow up regarding your request for assistance with the review. If you could kindly share the review you'd like me to rewrite, I would be more than happy to help!

Thank you for your consideration, and I look forward to your response.

Best regards,

[Your Name]
Bien sûr ! Veuillez fournir l'avis que vous souhaitez que je reformule.


In [ ]:
review = df.Review[5]
overall_chain(review)

**Repeat the above twice for different products or reviews**

## Router Chain

In [27]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [28]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [29]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [30]:
llm = ChatOpenAI(temperature=0)

In [31]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [32]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [33]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [34]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [35]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\Sadiq\AppData\Local\Temp\ipykernel_20852\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [36]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation and emits radiation at all frequencies. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This type of radiation is important in understanding concepts such as thermal radiation and the behavior of objects at different temperatures."

In [37]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'The answer to 2 + 2 is 4.'

In [38]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}
> Finished chain.


'Every cell in our body contains DNA because DNA is the genetic material that carries the instructions for the development, functioning, and reproduction of all living organisms. DNA contains the information needed to build and maintain an organism, including the proteins that make up our cells and tissues. \n\nHaving DNA in every cell ensures that each cell has the necessary genetic information to carry out its specific functions and to replicate itself accurately during cell division. This ensures that the genetic information is passed on to the next generation of cells. \n\nAdditionally, DNA is constantly being used by cells to carry out processes such as protein synthesis, cell division, and repair. Having DNA in every cell allows for the coordination of these processes and ensures the proper functioning of the organism as a whole.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**